## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [17]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../weather_database/WeatherPy_Database.csv")
city_data_df.loc[city_data_df["Country"]=="ES"]

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
129,129,Muros,ES,42.7762,-9.0603,62.10,70,23,16.53,few clouds
285,285,Los Llanos De Aridane,ES,28.6585,-17.9182,64.15,83,20,10.36,few clouds
329,329,Felanitx,ES,39.4696,3.1483,60.13,80,98,17.27,overcast clouds
466,466,Mao,ES,39.8885,4.2658,54.16,94,100,21.85,mist
531,531,Naron,ES,43.5167,-8.1528,59.56,66,30,7.83,scattered clouds
558,558,Santa Cruz De La Palma,ES,28.6835,-17.7642,68.14,68,20,10.36,few clouds


In [4]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 30
What is the maximum temperature you would like for your trip? 90


In [5]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
vac_city_data_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"]>= min_temp)].copy()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
0,0,Arraial Do Cabo,BR,-22.9661,-42.0278,87.76,58,0,16.11,clear sky
1,1,Portland,US,45.5234,-122.6762,48.42,82,100,7.00,overcast clouds
4,4,Cape Town,ZA,-33.9258,18.4232,67.12,76,75,11.50,broken clouds
5,5,Shadegan,IR,30.6497,48.6647,87.75,6,0,6.58,clear sky
6,6,Aranos,NaN,-24.1333,19.1167,77.74,47,77,14.20,broken clouds
...,...,...,...,...,...,...,...,...,...,...
658,658,Longonjo,AO,-12.9067,15.2533,72.18,80,97,3.04,light rain
660,660,Trairi,BR,-3.2778,-39.2689,85.82,63,47,12.17,scattered clouds
661,661,Kinango,KE,-4.1372,39.3153,84.31,65,84,10.89,broken clouds
662,662,Gombong,ID,-7.6072,109.5142,78.22,89,100,2.26,very heavy rain


In [6]:
# 4a. Determine if there are any empty rows.
vac_city_data_df.count()

City_ID        529
City           529
Country        524
Lat            529
Lng            529
Max Temp       529
Humidity       529
Cloudiness     529
Wind Speed     529
Description    529
dtype: int64

In [7]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
vac_city_data_df = vac_city_data_df.dropna()
vac_city_data_df.count()

City_ID        524
City           524
Country        524
Lat            524
Lng            524
Max Temp       524
Humidity       524
Cloudiness     524
Wind Speed     524
Description    524
dtype: int64

In [8]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = vac_city_data_df[["City", "Country", "Max Temp", "Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
0,Arraial Do Cabo,BR,87.76,clear sky,-22.9661,-42.0278,
1,Portland,US,48.42,overcast clouds,45.5234,-122.6762,
4,Cape Town,ZA,67.12,broken clouds,-33.9258,18.4232,
5,Shadegan,IR,87.75,clear sky,30.6497,48.6647,
7,Butaritari,KI,81.64,broken clouds,3.0707,172.7902,
8,Kavali,IN,82.04,clear sky,14.9167,79.9833,
9,Lebu,CL,55.54,few clouds,-37.6167,-73.6500,
11,Pacasmayo,PE,73.76,broken clouds,-7.4006,-79.5714,
12,Cauquenes,CL,61.77,clear sky,-35.9671,-72.3225,
13,Bluff,NZ,57.92,overcast clouds,-46.6000,168.3333,


In [9]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}


# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"   

    # 6e. Make request and retrieve the JSON data from the search. 
    hotel_data = requests.get(base_url, params = params).json()
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel_data["results"][0]["name"]
    except (IndexError):
        print('Hotel not found... skipping')
        

Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... s

In [10]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()


In [11]:
#Check dropped NaNs
print(hotel_df.count())
print(clean_hotel_df.count())

City           524
Country        524
Max Temp       524
Description    524
Lat            524
Lng            524
Hotel Name     524
dtype: int64
City           524
Country        524
Max Temp       524
Description    524
Lat            524
Lng            524
Hotel Name     524
dtype: int64


In [12]:
# 8a. Create the output File (CSV)
output_data_file = "./WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [13]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Description} with a current temp of {Max Temp} F</dd>
<dt> </dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]


In [14]:
# 11a. Add a marker layer for each city to the map. 
locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure

fig

Figure(layout=FigureLayout(height='420px'))